# 协程/coroutine

[PEP 342 -- Coroutines via Enhanced Generators](https://www.python.org/dev/peps/pep-0342/)

### Generator as coroutine

In [1]:
def simple_coroutine():
    print("-> coroutine started.")
    x = yield
    print("-> coroutine received:", x)

In [2]:
my_coro = simple_coroutine()

In [3]:
my_coro

<generator object simple_coroutine at 0x7fb690212cf0>

In [4]:
next(my_coro) # 预激（prime）

-> coroutine started.


In [5]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

In [6]:
import inspect
inspect.getgeneratorstate(my_coro)

'GEN_CLOSED'

需要先调用next(my_coro)来激活协程，或者使用send(None)。否则会发生以下错误：

In [7]:
my_coro = simple_coroutine()
my_coro.send(213)

TypeError: can't send non-None value to a just-started generator

一个更好的例子：

In [8]:
def simple_coro2(a):
    print("-> started: a =", a)
    b = yield a
    print("-> received: b =", b)
    c = yield a + b
    print("-> received: c =", c)

In [9]:
my_coro2 = simple_coro2(14)
from inspect import getgeneratorstate
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [10]:
next(my_coro2)

-> started: a = 14


14

In [11]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [12]:
my_coro2.send(28)

-> received: b = 28


42

In [13]:
my_coro2.send(99)

-> received: c = 99


StopIteration: 

In [14]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

使用协程计算平均值

In [25]:
def averager():
    total = 0.
    cnt = 0
    avg = None
    while True:
        term = yield avg
        total += term
        cnt += 1
        avg = total / cnt

In [26]:
avg_coro = averager()

In [27]:
next(avg_coro)

In [28]:
import random
randi = (random.randint(1, 50) for i in range(5))
for i in randi:
    print(i)
    print(avg_coro.send(i))

16
16.0
5
10.5
47
22.666666666666668
25
23.25
7
20.0


In [29]:
avg_coro.close()

In [30]:
from inspect import getgeneratorstate
getgeneratorstate(avg_coro)

'GEN_CLOSED'

### 使用装饰器来预激活协程
实现：[coroutine 装饰器](coroutil.py)

In [33]:
from coroutil import coroutine
@coroutine
def averager():
    total = 0.
    cnt = 0
    avg = None
    while True:
        term = yield avg
        total += term
        cnt += 1
        avg = total / cnt

In [34]:
avg = averager()
avg.send(10) # 可以立刻给协程发送消息

10.0

### 终止协程和异常处理

In [35]:
avg.send("fuck")

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [37]:
getgeneratorstate(avg) # 事实上，可以通过发送某个值来停止协程

'GEN_CLOSED'

生成器对象上有两个方法可以给我们调用：close() 和 throw()

In [40]:
class DemoException(Exception):
    pass

@coroutine
def demo_exc_handling():
    print('-> coro started')
    while True:
        try:
            x = yield
        except DemoException:
            print("handling DemoException, continuing")
        else:
            print("coro received:", x)
    raise RuntimeError("this line should never run.")
            

In [42]:
coro = demo_exc_handling()

-> coro started


In [43]:
coro.send(10)

coro received: 10


In [45]:
coro.throw(DemoException)

handling DemoException, continuing


In [46]:
coro.send(20)

coro received: 20


In [47]:
coro.close()

In [48]:
coro.send(10)

StopIteration: 

### 让协程返回值

为了让协程返回值，协程必须正常终止，比如说需要某个条件来退出无限循环。

In [54]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')

@coroutine
def averager():
    total = 0.
    cnt = 0
    avg = None
    while True:
        term = yield avg
        if term is None:
            break
        total += term
        cnt += 1
        avg = total / cnt
    return Result(cnt, avg)

In [55]:
avg = averager()
avg.send(10)

10.0

In [56]:
avg.send(20)

15.0

In [57]:
r = avg.send(None)

StopIteration: Result(count=2, average=15.0)

In [62]:
r

NameError: name 'r' is not defined

捕获StopIteration异常来获取其返回的值

In [63]:
avg = averager()
avg.send(10)
try:
    avg.send(None)
except StopIteration as exc:
    result = exc.value

In [64]:
result

Result(count=1, average=10.0)

### 使用yield from

yield from 可以用于简化 for 循环：

In [1]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i

def gen2():
    yield from 'AB'
    yield from range(1, 3)

In [3]:
list(gen()) == list(gen2())

True

用于链接可迭代对象：

In [4]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [5]:
s = 'ABC'
t = tuple(range(4))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2, 3]

- 委派生成器：包含 yield from <iterable> 表达式的生成器函数；
- 子生成器：从 yield from 表达式中 <iterable> 部分获取的生成器；
- 调用方：客户端。

In [108]:
# BEGIN YIELD_FROM_AVERAGER
from collections import namedtuple

Result = namedtuple('Result', 'count average')


# the subgenerator
def averager():  # <1>
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield  # <2>
        if term is None:  # <3>
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)  # <4>


# the delegating generator
def grouper(results, key):  # <5>
    while True:  # <6>
        results[key] = yield from averager()  # <7>


# the client code, a.k.a. the caller
def main(data):  # <8>
    results = {}
    for key, values in data.items():
        group = grouper(results, key)  # <9>
        next(group)  # <10>
        for value in values:
            group.send(value)  # <11>
        group.send(None)  # important! <12>

    # print(results)  # uncomment to debug
    report(results)


# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))


data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}


In [109]:
main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m
